 # Opis notatnika
Głównym celem w tym notatniku jest sprawdzenie i ewentualne dostosowanie struktury danych z plików źródłowych do formatu zgodnego z `Postgres`, a następnie wgranie ich na nasz serwer. Dzięki temu w późniejszych krokach możemy niezależnie użyć danych do analizy czy raportowania.

## Połączenie z bazą danych

##### Importujemy potrzebne biblioteki oraz plik z ukrytymi danymi do logowania

In [5]:
import psycopg2
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import text

In [6]:
# Ładujemy zmienne środowiskowe z pliku .env
load_dotenv('DB_pass.env')

True

##### Podłączamy się do bazy danych

In [8]:
# Tworzymy url połączenia
url = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('N_DB_NAME')}"

# Tworzymy engine dla sqlalchemy
engine = create_engine(url)

 # Załadowanie i wczytanie ramek do obszaru roboczego

##### Tworzymy funkcję `load_raw_data`, która przyjmuje jeden parametr `file_name`, czyli nazwę pliku do zaczytania. Jej zadaniem jest wczytanie surowego pliku, zmodyfikowanie nazw kolumn z `NAZWA_KOLUMNY` na `nazwa_kolumny` oraz zwrócenie tak zmodyfikowanej ramki danych.

In [11]:
def load_raw_data(file_name):
    df = pd.read_csv(file_name)
    df.columns = df.columns.str.lower()
    return df

##### Zaczytujemy poszczególne pliki do ramek i sprawdzamy typy danych, czy pokrywają się z bazą danych w postgresql

In [13]:
baza1 = load_raw_data('../data/processed/TEST_BAZA1_CLEANED.csv')
baza1.head()

,id,number,end_dt,segment
0,12373110,NaN,14.03.2024 10:46:02,Soho
1,12373110,NaN,13.03.2024 19:48:21,Soho
2,12370190,NaN,25.06.2017 11:58:41,Soho
3,12370190,NaN,29.10.2022 01:20:34,Soho
4,12370190,NaN,29.10.2022 02:20:33,Soho


In [14]:
baza1.dtypes

id           int64
number     float64
end_dt      object
segment     object
dtype: object

In [15]:
# Konwersja kolumny 'END_DT' na datetime z określeniem konkretnego formatu
baza1['end_dt'] = pd.to_datetime(baza1['end_dt'], format='%d.%m.%Y %H:%M:%S', errors='coerce')

# Sprawdzenie wyników konwersji
print(baza1['end_dt'].head())

0   2024-03-14 10:46:02
1   2024-03-13 19:48:21
2   2017-06-25 11:58:41
3   2022-10-29 01:20:34
4   2022-10-29 02:20:33
Name: end_dt, dtype: datetime64[ns]


In [16]:
# Konwersja kolumny 'NUMBER' na ciąg znaków, ponieważ kolumna reprezentuje kategorię/identyfikator, a nie wartości liczbowe.
baza1['number'] = baza1['number'].astype(str)

# Usunięcie ".0" z końca liczby w formacie string
baza1['number'] = baza1['number'].str.replace(r'\.0$', '', regex=True)

# Sprawdzenie wyników
print(baza1.dtypes)

id                  int64
number             object
end_dt     datetime64[ns]
segment            object
dtype: object


In [17]:
baza2 = load_raw_data('../data/processed/TEST_BAZA2_CLEANED.csv')
baza2.head()

,id,number,plan
0,10886857,531778955,S
1,10886857,500830441,S
2,10886857,599709277,S
3,10886857,580058812,S
4,10886857,542422081,S


In [18]:
baza2.dtypes

id         int64
number     int64
plan      object
dtype: object

In [19]:
# Konwersja kolumny 'NUMBER' na ciąg znaków, ponieważ kolumna reprezentuje kategorię/identyfikator, a nie wartości liczbowe.
baza2['number'] = baza2['number'].astype(str)

# Sprawdzenie wyników
print(baza2.dtypes)

id         int64
number    object
plan      object
dtype: object


 # Eksport danych na bazę

##### Tworzymy funkcję `export_table_to_db`, która przyjmuje dwa parametry "df" i "table_name", czyli nazwę wczytywanej ramki danych i tabeli docelowej w bazie postgresql.

In [22]:
def export_table_to_db(df, table_name):
    print(f"Loading data into {table_name}...")

    # Eksportujemy ramkę do bazy danych
    df.to_sql(
        name=table_name,
        con=engine,
        if_exists='append',
        index=False,
        chunksize=1000
    )

    print(f"Data loaded successfully into {table_name}.")

 ## Wgrywanie danych

 ### Wgranie `baza1` do tabeli `baza1`

In [25]:
export_table_to_db(baza1, 'baza1')

Loading data into baza1...
Data loaded successfully into baza1.


 ### Wgranie `baza2` do tabeli `baza2`

In [27]:
export_table_to_db(baza2, 'baza2')

Loading data into baza2...
Data loaded successfully into baza2.


In [28]:
# Sprawdzenie, czy kolumna NUMBER ma prawidłowo wczytany NULL zamiast nan
null_check = pd.read_sql("SELECT * FROM baza1 WHERE NUMBER IS NULL;", con=engine)
print(null_check)


Empty DataFrame
Columns: [id, number, end_dt, segment]
Index: []


In [29]:
# Definiujemy zapytanie SQL, które zamienia wartości 'nan' na NULL w bazie danych
update_query = text("""
    UPDATE baza1
    SET number = NULL
    WHERE number = 'nan';
""")

# Wykonanie zapytania na bazie danych
with engine.begin() as connection:
    connection.execute(update_query)

In [30]:
# Sprawdzenie, czy kolumna 'number' ma teraz NULL zamiast 'nan'
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM baza1 WHERE number IS NULL"))
    print(result.fetchall())

[(12373110, None, datetime.datetime(2024, 3, 14, 10, 46, 2), 'Soho'), (12373110, None, datetime.datetime(2024, 3, 13, 19, 48, 21), 'Soho'), (12370190, None, datetime.datetime(2017, 6, 25, 11, 58, 41), 'Soho'), (12370190, None, datetime.datetime(2022, 10, 29, 1, 20, 34), 'Soho'), (12370190, None, datetime.datetime(2022, 10, 29, 2, 20, 33), 'Soho'), (12370190, None, datetime.datetime(2022, 10, 29, 5, 50, 57), 'Soho'), (12370190, None, datetime.datetime(2022, 10, 29, 16, 15, 39), 'Soho'), (12370190, None, datetime.datetime(2022, 10, 29, 4, 0, 13), 'Soho'), (12370190, None, datetime.datetime(2022, 10, 29, 1, 6, 15), 'Soho'), (12370190, None, datetime.datetime(2022, 10, 29, 6, 36, 43), 'Soho'), (12370190, None, datetime.datetime(2022, 10, 29, 0, 35, 34), 'Soho'), (12370190, None, datetime.datetime(2022, 10, 29, 14, 28, 48), 'Soho'), (12370190, None, datetime.datetime(2022, 10, 29, 22, 54, 11), 'Soho'), (12370190, None, datetime.datetime(2022, 10, 29, 11, 45, 42), 'Soho'), (12370190, None, d

 # Podsumowanie
 W tym notatniku załadowaliśmy pobrane wcześniej pliki na bazę danych. Dzięki temu stworzyliśmy centralne miejsce ich magazynowania, co wykorzystamy zarówno przy analizie danych, jak i przy późniejszej budowie systemu raportowego.

In [32]:
msg = "Wszystko wygląda OK :) Przechodzimy do kolejnego zadania."
print(msg)

Wszystko wygląda OK :) Przechodzimy do kolejnego zadania.
